In [ ]:
import contextlib
import json
import os 
import cv2
import pandas as pd
import numpy as np
from PIL import Image
from collections import defaultdict
os.chdir("/content/drive/MyDrive/CvipCodeFolder/Shared Dataset/Code")
from utils import *

In [ ]:
!pip install utils 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def convert_vott_json(name, files, img_path):
    # Create folders
    path = make_dirs()
    name = os.path.join(path,name)
    print(name)
    # Import json
    data = []
    for file in glob.glob(files):
        with open(file) as f:
            jdata = json.load(f)
            jdata['json_file'] = file
            data.append(jdata)

    # Get all categories
    file_name, wh, cat = [], [], []
    for i, x in enumerate(tqdm(data, desc='Files and Shapes')):
        with contextlib.suppress(Exception):
            cat.extend(a['tags'][0] for a in x['regions'])  # categories

    # Write *.names file
    names = sorted(pd.unique(cat))
    with open(name + '.names', 'a') as file:
        [file.write('%s\n' % a) for a in names]

    # Write labels file
    n1, n2 = 0, 0
    missing_images = []
    for i, x in enumerate(tqdm(data, desc='Annotations')):

        f = glob.glob(img_path + '/' +x['asset']['name'])
        if len(f):
            f = f[0]
            file_name.append(f)
            wh = exif_size(Image.open(f))  # (width, height)

            n1 += 1
            if (len(f) > 0) and (wh[0] > 0) and (wh[1] > 0):
                n2 += 1

                # append filename to list
                with open(name + '.txt', 'a') as file:
                    file.write('%s\n' % f)

                # write labelsfile
                label_name = Path(f).stem + '.txt'
                with open(os.path.join(path,'labels',label_name), 'a') as file:
                    for a in x['regions']:
                        category_id = names.index(a['tags'][0])

                        # The INFOLKS bounding box format is [x-min, y-min, x-max, y-max]
                        box = a['boundingBox']
                        box = np.array([box['left'], box['top'], box['width'], box['height']]).ravel()
                        box[[0, 2]] /= wh[0]  # normalize x by width
                        box[[1, 3]] /= wh[1]  # normalize y by height
                        box = [box[0] + box[2] / 2, box[1] + box[3] / 2, box[2], box[3]]  # xywh

                        if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
                            file.write('%g %.6f %.6f %.6f %.6f\n' % (category_id, *box))
        else:
            missing_images.append(x['asset']['name'])

    print('Attempted %g json imports, found %g images, imported %g annotations successfully' % (i, n1, n2))
    if len(missing_images):
        print('WARNING, missing images:', missing_images)

    # Split data into train, test, and validate files
    split_files(name, file_name)
    print(f'Done. Output saved to {os.path.join(os.getcwd(),path)}')





    

In [ ]:
convert_vott_json("data","/content/drive/MyDrive/CvipCodeFolder/assets/*.json","/content/drive/MyDrive/CvipCodeFolder/images")


new_dir/data


Annotations: 100%|██████████| 270/270 [00:54<00:00,  4.94it/s]

Attempted 269 json imports, found 270 images, imported 270 annotations successfully
Done. Output saved to /content/drive/.shortcut-targets-by-id/1yvjEDSUpxeukrIX-hwWVbrAIUrLlLGg5/Shared Dataset/Code/new_dir


In [ ]:
import shutil

In [ ]:
for files in os.listdir("/content/drive/MyDrive/cvip/new_dir/labels"):
  shutil.copy(os.path.join("/content/drive/MyDrive/cvip/new_dir/labels",files),"/content/drive/MyDrive/imges")